# Setup
  - Install any required packages (use the `set_up.sh` script)
  - Set up all of the imports
  - set up the random seed

In [ ]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as sched
import torch.utils.data as data
import urllib3
import os
import shutil
import json
import ujson
import pandas as pd

In [ ]:
rand_seed = 3716 # setting the random seed for consistent runs

random.seed(rand_seed)
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
torch.cuda.manual_seed_all(rand_seed)

# Data Processing
This section is to import and pre-process the data

In [ ]:
import_data = False
# if this flag is set to true, then the data will be downloaded
# otherwise this step will be skipped

In [ ]:
def data_download(url="https://google.com/", file_name="default.json", file_path="/tmp/"):
    with http.request('GET', url, preload_content=False) as r, open((file_path + file_name), 'wb') as out_file:       
            shutil.copyfileobj(r, out_file)
    return ("the", file_name, "was downloaded to", file_path)

In [ ]:
GEN_DATA_PATH = os.getcwd() + "/data_v2/" 

TRAIN_DATA_PATH = GEN_DATA_PATH + "train/" 
DEV_DATA_PATH = GEN_DATA_PATH + "dev/"


if import_data:
    
    print("LOG: data import started")

    SQUAD_DATA_TRAINING = "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json"
    SQUAD_DATA_DEV = "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json"
    
    http = urllib3.PoolManager()
    
    print("LOG: starting train data download process")
    data_download(url=SQUAD_DATA_TRAINING, file_name="train_raw_data.json", file_path=TRAIN_DATA_PATH)
    print("LOG: starting dev data download process")
    data_download(url=SQUAD_DATA_DEV, file_name="dev_raw_data.json", file_path=DEV_DATA_PATH)
    
    print("LOG: finished data importing")
    


Now that we have the data downloaded, we need to do prepare it for training. Luckily, we don't need too much for this datatset.

In [ ]:
dev = pd.read_json('data_v2/dev/dev_raw_data.json')
train = pd.read_json('data_v2/train/train_raw_data.json')

In [ ]:
print("dev raw shape:", dev.shape)
print("train raw shape:", train.shape)

In [ ]:
# we only care about the data itself at this point 
# since we know it is version 2

train = train['data']
dev = dev['data']

In [ ]:
contexts = []
questions = []
answers_text = []
answers_starti = []
impossibility = []
topics = []

In [ ]:
def cleaner(qac, context, title):
    pass

In [ ]:
def process_data(dataset):
    for topic in dataset:
        print(topic['title'])
        for paragraph in topic['paragraphs']:
            for qac in paragraph['qas']:
                
                topics.append(topic['title'])
                questions.append(qac['question'])
                if qac['is_impossible'] == "False":
                    answers_text.append(qac['answers'][0]['text']) 
                    answers_starti.append(qac['answers'][0]['answer_start'])
                    impossibility.append(False)

                else:
                    answers_text.append("")
                    answers_starti.append("")
                    impossibility.append(True)
                
                contexts.append(paragraph['context'])
                
    assembled_df = pd.DataFrame({"context":contexts, 
                                 "question": questions, 
                                 "answer_start": answers_starti, 
                                 "answers_text": answers_text,
                                 "impossibility": impossibility,
                                 "topic": topics})
    
    return assembled_df

In [ ]:
assembled_df = process_data(train)

In [ ]:
print(assembled_df.shape)

In [1]:
# TODO: 
# - clean the data via the cleaner function
# - define the custom pytorch data structure
# - add word/vector embeddings
# - implement a basic model

# Models

Create the different models that will be used as a part of the baseline training.

# Training

This section will contain all code used in training